# Exploration and Debugging <br> 
### The objectives of this notebook are: 
* **Load** a sample data file from either the regular season or playoff;
* **Understand** an initial overview of the data structure;
* **Create** an ipywidget to interact with a few values of a particular game.

## Imports

In [2]:
import json
import ipywidgets as widgets
import pandas as pd


## Sample Data Loading

In [3]:
data_path = '../data/raw/'

## Exploration of the data structure

## Which side do team play when they are at home?

In [15]:
df = pd.read_csv("../ift6758/data/tidy_data.csv")
df.head(10)

,game_id,game_time,period,period_time,team,shooter,goalie,is_goal,shot_type,x_coordinate,y_coordinate,is_empty_net,strength,is_playoff,is_home_team,shot_distance
0,2019020910,0:15,1,00:15,Ottawa Senators,Tyler Ennis,Anton Khudobin,False,Backhand,81.0,-7.0,NaN,NaN,False,True,10.630146
1,2019020910,0:18,1,00:18,Ottawa Senators,Thomas Chabot,Anton Khudobin,False,Wrist Shot,50.0,38.0,NaN,NaN,False,True,54.451814
2,2019020910,0:34,1,00:34,Ottawa Senators,Dylan DeMelo,Anton Khudobin,False,Wrist Shot,32.0,-32.0,NaN,NaN,False,True,65.368188
3,2019020910,1:11,1,01:11,Ottawa Senators,Anthony Duclair,Anton Khudobin,False,Wrist Shot,59.0,29.0,NaN,NaN,False,True,41.725292
4,2019020910,1:43,1,01:43,Ottawa Senators,Cody Goloubef,Anton Khudobin,False,Wrist Shot,33.0,-20.0,NaN,NaN,False,True,59.464275
5,2019020910,2:24,1,02:24,Ottawa Senators,Scott Sabourin,Anton Khudobin,False,Wrist Shot,65.0,17.0,NaN,NaN,False,True,29.410882
6,2019020910,2:56,1,02:56,Dallas Stars,Jason Robertson,Craig Anderson,False,Slap Shot,-51.0,-33.0,NaN,NaN,False,False,50.328918
7,2019020910,3:03,1,03:03,Dallas Stars,John Klingberg,Craig Anderson,True,Deflected,-39.0,30.0,False,Even,False,False,58.309519
8,2019020910,3:41,1,03:41,Ottawa Senators,Brady Tkachuk,Anton Khudobin,False,Deflected,83.0,7.0,NaN,NaN,False,True,9.219544
9,2019020910,3:58,1,03:58,Ottawa Senators,Jean-Gabriel Pageau,Anton Khudobin,False,Wrist Shot,44.0,-29.0,NaN,NaN,False,True,53.535035


In [18]:
df = df[df['is_home_team'] == True]
df = df[df['is_goal']==True]
df = df[df['period']==1]
df.groupby('team')['x_coordinate'].mean()

team
Anaheim Ducks            71.581250
Arizona Coyotes          64.540541
Boston Bruins            66.641791
Buffalo Sabres           69.404580
Calgary Flames          -67.791209
Carolina Hurricanes     -72.017544
Chicago Blackhawks      -70.256545
Colorado Avalanche      -58.556150
Columbus Blue Jackets    68.077348
Dallas Stars            -69.012422
Detroit Red Wings       -68.924242
Edmonton Oilers          71.633663
Florida Panthers        -67.212644
Los Angeles Kings       -70.839695
Minnesota Wild          -70.602273
Montréal Canadiens      -69.704545
Nashville Predators      70.111111
New Jersey Devils        69.758170
New York Islanders       68.310881
New York Rangers        -74.066667
Ottawa Senators          69.952381
Philadelphia Flyers     -66.829932
Pittsburgh Penguins      70.171296
San Jose Sharks          70.429268
St. Louis Blues          68.563536
Tampa Bay Lightning      59.294118
Toronto Maple Leafs     -71.027027
Vancouver Canucks       -69.513333
Vegas Golden Kn

## Widget Interactions

In [6]:
from IPython.display import display
w = widgets.IntSlider()
display(w)

IntSlider(value=0)